## TheMealDB API — Endpoint + Parameter Setup

https://www.themealdb.com/api.php

We are going to use TheMealDB API, and we will not manually attach `?something=` at the end of URLs.  
Instead, we pass parameters through the `params={}` argument in `requests.get()`.

### Section 1: Endpoints we will use:

| Task | Endpoint URL | Required Parameter | Meaning |
|---|---|---|---|
| Search meals by name text | `https://www.themealdb.com/api/json/v1/1/search.php` | `s` | string to search inside meal names |
| Filter meals by category | `https://www.themealdb.com/api/json/v1/1/filter.php` | `c` | exact category name (Example: `Dessert`) |
| Lookup full details by ID | `https://www.themealdb.com/api/json/v1/1/lookup.php` | `i` | the exact `idMeal` value |

> **Important:** `q=` does not work here — TheMealDB does not use `q`.  
> These 3 parameter keys — `s`, `c`, `i` — are the official ones.




### Why we do it this way
APIs require exact field names.  
LLMs may guess parameter names — but APIs do not guess.  
So we always check parameter directly in documentation.


**Exampple Prompt to LLM:**

I want to find meals that are desserts. I know TheMealDB has multiple endpoints and each endpoint uses a different parameter name.

The endpoints and required parameters are:

1. https://www.themealdb.com/api/json/v1/1/search.php: "s"
2. https://www.themealdb.com/api/json/v1/1/filter.php: "c"
3. https://www.themealdb.com/api/json/v1/1/lookup.php: "i"


Which endpoint and parameter should I call to retrieve a list of Dessert meals?

Do NOT give me the meals yet.  
Just tell me which endpoint + parameter to call, in the format:

endpoint: <…>
params: { "<key>": "<value>" }

I will run the API call myself and paste the result back to you.


**Possible LLM answer:**

endpoint: /filter.php  
params: { "c": "dessert" }


In [1]:
import requests

url = "https://www.themealdb.com/api/json/v1/1/filter.php"   # endpoint
params = { "c": "dessert" }                                  # search term

r = requests.get(url, params=params)
dessert = r.json()

dessert



{'meals': [{'strMeal': 'Æbleskiver', 'strMealThumb': None, 'idMeal': '53120'},
  {'strMeal': 'Anzac biscuits',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/q47rkb1762324620.jpg',
   'idMeal': '53111'},
  {'strMeal': 'Apam balik',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/adxcbq1619787919.jpg',
   'idMeal': '53049'},
  {'strMeal': 'Apple & Blackberry Crumble',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/xvsurr1511719182.jpg',
   'idMeal': '52893'},
  {'strMeal': 'Apple Frangipan Tart',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/wxywrq1468235067.jpg',
   'idMeal': '52768'},
  {'strMeal': 'Bakewell tart',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/wyrqqq1468233628.jpg',
   'idMeal': '52767'},
  {'strMeal': 'Banana Pancakes',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/sywswr1511383814.jpg',
   'idMeal': '52855'},
  {'strMeal': 'Battenberg Cake',
   'strMealThumb

### Tool Use Exercise  — “Find meals that contain shrimp (but try more than one method)”

Goal:
We want to find meals that contain **shrimp** (or “prawn”) — BUT TheMealDB does not have a direct “ingredient search” endpoint.  
So the LLM must propose ways to *approximate* this.

We will ask the LLM for NOT just one approach — but at least **two different** possible tool-use strategies.

**Prompt to LLM:**

I want to find meals that contain shrimp. TheMealDB does not have a direct `ingredient` search endpoint.  
Give me **two different** possible approaches I could take using TheMealDB endpoints to identify shrimp meals.

Rules:
- You must reference actual endpoints that exist (like `/search.php`, `/filter.php`, `/lookup.php`).
- For each approach, tell me which endpoint I should call FIRST, and what the parameter should be.
- Do NOT guess any meal names.  
- Do NOT claim you know which meals contain shrimp.  
- Only propose the *strategy*.


**After the LLM answers:**

→  Choose one approach  
→  Write the code to make that first API call  
→  Paste the resulting JSON **back into the LLM** and ask:

"Does this result still support the strategy? What do I do next?"

**Optional follow-up prompt ( ask LLM):**

This is the only approach you gave.  
Can you propose a *different* approach using a different endpoint flow?  
Again — do not tell me the answer directly — only propose the tool path.

→ Pick the second approach and test it as well.

---

**Note:**
The learning goal here is:  
The LLM is not allowed to “know the shrimp meals”.  
It is only allowed to propose *plans* to discover them using the tools (API calls).


In [2]:
## METHOD 1
## CODE:
import requests
import pandas as pd

base_url = "https://www.themealdb.com/api/json/v1/1/search.php"

params = {"s": "shrimp"} 
response = requests.get(base_url, params=params)
data = response.json()
meals = data.get("meals", [])
print(f"Returned {len(meals)} meals.\n")

if meals:
    df_search = pd.DataFrame(meals)[["idMeal", "strMeal", "strCategory", "strArea"]]
    display(df_search.head(5))
else:
    print("No meals found.")

Returned 1 meals.



,idMeal,strMeal,strCategory,strArea
0,52953,Shrimp Chow Fun,Seafood,Chinese


In [3]:
## METHOD 2
## CODE:
import requests
import pandas as pd

filter_url = "https://www.themealdb.com/api/json/v1/1/filter.php"
lookup_url = "https://www.themealdb.com/api/json/v1/1/lookup.php"
filter_params = {"c": "Seafood"}
resp = requests.get(filter_url, params=filter_params)
seafood_data = resp.json()
seafood_meals = seafood_data.get("meals", [])
print(f"Found {len(seafood_meals)} meals in the Seafood category.\n")

verified = []
for m in seafood_meals[:10]: 
    meal_id = m["idMeal"]
    lookup = requests.get(lookup_url, params={"i": meal_id}).json()
    full = lookup.get("meals", [])[0]
    ingredients = [
        full.get(f"strIngredient{i}") for i in range(1, 21)
        if full.get(f"strIngredient{i}")
    ]
    if any("shrimp" in ing.lower() or "prawn" in ing.lower() for ing in ingredients):
        verified.append({
            "idMeal": meal_id,
            "strMeal": full["strMeal"],
            "strCategory": full["strCategory"],
            "strArea": full["strArea"],
            "ingredients": ", ".join(ingredients)
        })
df_verified = pd.DataFrame(verified)
display(df_verified.head(10))

Found 32 meals in the Seafood category.



,idMeal,strMeal,strCategory,strArea,ingredients
0,52802,Fish pie,Seafood,British,"Floury Potatoes, Olive Oil, Semi-skimmed Milk,..."
1,52918,Fish Stew with Rouille,Seafood,French,"Prawns, Olive Oil, Dry White Wine, Fish Stock,..."
2,52764,Garides Saganaki,Seafood,Greek,"Raw king prawns, Olive oil, Chopped onion, Fre..."


## Section 2: How to Use an LLM respectfully

The LLM is **not** here to replace your thinking.

The LLM is here to:
- suggest *ideas*
- help you *communicate patterns*
- help you *frame claims that can be verified*

The LLM is **not allowed** to invent facts, ingredients, or data that the API did not provide.

### GOLDEN RULE

**LLM = language reasoning + framing**  
**API = actual facts / evidence**

We ALWAYS verify LLM claims against real data.

---

### Good LLM use examples

| Good (allowed) | Why |
|---|---|
| “Here are 2 ways to describe the difference between these meals.” | helping with communication |
| “Can you generate 2 testable claims that I can check in pandas?” | helping generate hypotheses |
| “Summarize the evidence table I pasted.” | summarizing what already exists |

### Not allowed

| Not allowed | Why |
|---|---|
| “Tell me the ingredients for this meal.” | API already gives this — model might hallucinate |
| “Decide which meal is safe without seeing the data.” | decision must be data-driven |
| “Guess which meals have peanuts.” | guessing = not acceptable |

---

### The pattern you must follow

1) **API → get real data**
2) **YOU → tidy & prepare it**
3) **YOU → paste only the relevant subset into the LLM**
4) **LLM → proposes claims (not answers)**
5) **YOU → verify each claim in code**

This is the professional workflow in data work:
> models generate possible patterns → humans verify with real data.

---

### If the LLM says something, that is just a *claim*.  
It is not true until we verify it.

We trust the API values — not the LLM wording.


## Using the endpoints with your own question + your own purpose

For section 2, you are not just asking the LLM to give you answers  
You are using proper LLM prompts and API calls as *tools* to support a real question that YOU care about.

Examples (you don’t have to pick these):
- meals you might want to cook this week(ingredients, regions, etc.)
- meals that match your personal taste (sweet, spicy, seafood, regions, etc.)
- meals that fit a budget or fit your macros (low ingredient count, high protein)
- meals that avoid something (allergy, dislike, dietary restriction)

The point is: **you decide what makes a meal “good” or “interesting.”**  
Then you use the LLM prompts ans API calls as tools to explore that idea.

---

### The 3 endpoints

| What you want to do | Endpoint | Parameter |
|---|---|---|
| search by name text | `/search.php` | `s` |
| filter by category | `/filter.php` | `c` |
| lookup full detail for a single meal | `/lookup.php` | `i` |

---

There are also some other endpoints that TheMealDB API provides, you may wanna check them out and see if they can be helpful for solving your question.
- See here: https://www.themealdb.com/api.php

### What we are practicing today

1) **Ask the LLM** what API call(s) would help answer your question  
   → the LLM suggests a *plan*, not the answer

2) **YOU run the API call(s)** in Python  
   → get the real data

3) **Ask the LLM again** what patterns might be interesting  
   → suggestions, not answers

4) **YOU verify** any claims with real code  
   → API data is the ground truth

**In this lab — The LLM helps with planning, not with answering.**



## What LLM Assistance *Is* Allowed for Code

You **may** ask the LLM to help you write code for:
- filtering ingredients based on your allergy / dislike tokens
- counting ingredients
- grouping meals by a flag (safe / not-safe)
- formatting results (DataFrame, dict, list, etc.)

Example of *allowed* ask:

> “Here is my ingredients DataFrame. How do I filter rows where `ingredient_norm` contains the token `peanut`?”


### But important: the LLM is ONLY allowed to write code based on the data YOU already pasted in the chat, bacause otherwise it may come up keys and parameters that don't exit, and the codes might not work.

---

## What is NOT allowed

- asking the LLM to invent ingredients
- asking the LLM to decide which meals are safe (without code)
- asking the LLM to tell you the answer directly
- letting the LLM write code that assumes columns that don’t exist

Example of **not allowed** ask:

> “Which meals contain peanuts? Just tell me.”  
❌ → That skips the verification/data responsibility step.

---

The *results* must always be produced by code that runs on the real API data — not by the model’s text.


In [8]:
## CODE:
import pandas as pd
import requests

try:
    df_ingredients
except NameError:
    print("No existing DataFrame found — creating a small one from TheMealDB API.\n")
    resp = requests.get("https://www.themealdb.com/api/json/v1/1/filter.php?c=Seafood").json()
    sample_meals = resp.get("meals", [])[:5]

    rows = []
    for m in sample_meals:
        lookup = requests.get("https://www.themealdb.com/api/json/v1/1/lookup.php", params={"i": m["idMeal"]}).json()
        meal = lookup.get("meals", [])[0]
        ingredients = [
            meal.get(f"strIngredient{i}") for i in range(1, 21)
            if meal.get(f"strIngredient{i}")
        ]
        rows.append({
            "meal": meal["strMeal"],
            "ingredient_norm": [ing.lower() for ing in ingredients]
        })

    df_ingredients = pd.DataFrame(rows)
display(df_ingredients.head())
avoid_token = "peanut"
def contains_token(ingredient_list, token):
    """Return True if the token appears in any ingredient (case-insensitive)."""
    return any(token.lower() in ing.lower() for ing in ingredient_list)

df_flagged = df_ingredients.copy()
df_flagged["contains_peanut"] = df_flagged["ingredient_norm"].apply(
    lambda ings: contains_token(ings, avoid_token)
)
filtered = df_flagged[df_flagged["contains_peanut"] == True]
print(f"\nMeals containing '{avoid_token}': {len(filtered)}")
display(filtered)

,meal,ingredient_norm
0,Baked salmon with fennel & tomatoes,"[fennel, parsley, lemon, cherry tomatoes, oliv..."
1,Barramundi with Moroccan spices,"[barramundi, ground cumin, coriander, paprika,..."
2,Cajun spiced fish tacos,"[cajun, cayenne pepper, white fish, vegetable ..."
3,Escovitch Fish,"[red snapper, vegetable oil, garlic, ginger, t..."
4,Fish fofos,"[haddock, potatoes, green chilli, coriander, c..."



Meals containing 'peanut': 0


,meal,ingredient_norm,contains_peanut
